In [27]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

from sklearn.model_selection import train_test_split 

# Run Queries

In [28]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'SPO'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=20)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 1

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [29]:
clear_buyers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",138674,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [30]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",48322,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
frida_products = frida_products[['upc','id_tarjeta','categoria','subcategoria']].copy()

df_margen = get_fresh_query_result("https://internal-redash.federate.frubana.com/",144493,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_margen = pd.merge(df_margen, frida_products,  how='left', left_on=['sku'], right_on = ['upc'])
df_margen[["gmv_value","cogs_value"]] = df_margen[["gmv_value","cogs_value"]].astype(float)

200
200


In [31]:
df_margen['day'] = pd.to_datetime(df_margen['day'])
max_index = df_margen.groupby('id_tarjeta')['day'].idxmax()
df_margen = df_margen.loc[max_index].copy()

In [32]:
# calculate the profit margin by sku
df_margen2 = df_margen.groupby(by=['region','id_tarjeta']).apply(lambda x: (x['gmv_value'].sum() - x['cogs_value'].sum()) / x['gmv_value'].sum() if x['gmv_value'].sum() != 0 else 0).reset_index(name='GM')

# calculate the profit margin by subcategory
df_margen3 = df_margen.groupby(by=['region','subcategoria']).apply(lambda x: (x['gmv_value'].sum() - x['cogs_value'].sum()) / x['gmv_value'].sum()).reset_index(name='GM')

/home/jupyter-felipe.guevara/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [33]:
query = """
WITH exploded_data AS (
SELECT DISTINCT
    bs.site_identifier_value AS region_code,
    bo.offer_id,
    MAX(bcs.customer_segment_id) AS customer_segment_id_test,
    --bcs.customer_segment_name,
    MAX(bcs2.customer_segment_id) AS customer_segment_id_control,
    --bcs2.customer_segment_name,
    bo.offer_name AS discount,
    DATE(bo.start_date) AS start_date,
    DATE(bo.end_date) AS end_date,
    bo.offer_value::float as Value_Discount,
    replace(
        substring(
          boic.order_item_match_rule, 
          position('["' in boic.order_item_match_rule) + 2, 
          position('"]' in boic.order_item_match_rule) - position('["' in boic.order_item_match_rule) - 2
    ), '"', '') AS values_string
    
FROM postgres_broadleaf_federate."broadleaf.blc_offer"                          bo
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_offer_customer_seg_xref"   bocsx   ON (bo.offer_id = bocsx.offer_id AND ((bocsx.archived is NULL OR bocsx.archived='N')))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_tar_crit_offer_xref"      btcofx  ON (btcofx.offer_id = bo.offer_id)
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_offer_item_criteria"      boic    ON (btcofx.offer_item_criteria_id = boic.offer_item_criteria_id and (boic.archived is null or boic.archived ='N'))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     bs      ON bs.site_id = bo.catalog_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs     ON bocsx.customer_segment_id = bcs.customer_segment_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs2    ON replace(bcs.customer_segment_name,'TEST_LL2','CONTROL_LL2') = bcs2.customer_segment_name

WHERE offer_name ILIKE 'Hook_Localizacion%'
    AND end_date >= DATE_TRUNC('year', date(getdate()) - interval '0 year')
    AND end_date >= DATE_TRUNC('month', date(getdate()) - interval '0 month')
    --AND start_date >= DATE_TRUNC('month', date(getdate()) - interval '0 month')
    AND bo.offer_discount_type = 'PERCENT_OFF'
    AND bo.offer_type = 'ORDER_ITEM'
    AND bo.sndbx_id is NULL
    AND (bo.archived is NULL OR bo.archived='N')
    AND (bo.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
    AND bo.automatically_added = 'true'
    AND bo.offer_id <> -14489998
    --AND bs.site_identifier_value = 'BOG'
GROUP BY 1,2,5,6,7,8,9
), 
 
number_series AS (
  SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10
  UNION ALL SELECT 11 UNION ALL SELECT 12 UNION ALL SELECT 13 UNION ALL SELECT 14 UNION ALL SELECT 15 UNION ALL SELECT 16 UNION ALL SELECT 17 UNION ALL SELECT 18 UNION ALL SELECT 19 UNION ALL SELECT 20
  UNION ALL SELECT 21 UNION ALL SELECT 22 UNION ALL SELECT 23 UNION ALL SELECT 24 UNION ALL SELECT 25 UNION ALL SELECT 26 UNION ALL SELECT 27 UNION ALL SELECT 28 UNION ALL SELECT 29 UNION ALL SELECT 30
  UNION ALL SELECT 31 UNION ALL SELECT 32 UNION ALL SELECT 33 UNION ALL SELECT 34 UNION ALL SELECT 35 UNION ALL SELECT 36 UNION ALL SELECT 37 UNION ALL SELECT 38 UNION ALL SELECT 39 UNION ALL SELECT 40
  UNION ALL SELECT 41 UNION ALL SELECT 42 UNION ALL SELECT 43 UNION ALL SELECT 44 UNION ALL SELECT 45 UNION ALL SELECT 46 UNION ALL SELECT 47 UNION ALL SELECT 48 UNION ALL SELECT 49 UNION ALL SELECT 50
  UNION ALL SELECT 51 UNION ALL SELECT 52 UNION ALL SELECT 53 UNION ALL SELECT 54 UNION ALL SELECT 55 UNION ALL SELECT 56 UNION ALL SELECT 57 UNION ALL SELECT 58 UNION ALL SELECT 59 UNION ALL SELECT 60
  UNION ALL SELECT 61 UNION ALL SELECT 62 UNION ALL SELECT 63 UNION ALL SELECT 64 UNION ALL SELECT 65 UNION ALL SELECT 66 UNION ALL SELECT 67 UNION ALL SELECT 68 UNION ALL SELECT 69 UNION ALL SELECT 70
  UNION ALL SELECT 71 UNION ALL SELECT 72 UNION ALL SELECT 73 UNION ALL SELECT 74 UNION ALL SELECT 75 UNION ALL SELECT 76 UNION ALL SELECT 77 UNION ALL SELECT 78 UNION ALL SELECT 79 UNION ALL SELECT 80
  UNION ALL SELECT 81 UNION ALL SELECT 82 UNION ALL SELECT 83 UNION ALL SELECT 84 UNION ALL SELECT 85 UNION ALL SELECT 86 UNION ALL SELECT 87 UNION ALL SELECT 88 UNION ALL SELECT 89 UNION ALL SELECT 90
  UNION ALL SELECT 91 UNION ALL SELECT 92 UNION ALL SELECT 93 UNION ALL SELECT 94 UNION ALL SELECT 95 UNION ALL SELECT 96 UNION ALL SELECT 97 UNION ALL SELECT 98 UNION ALL SELECT 99 UNION ALL SELECT 100
)

SELECT DISTINCT
    ed.region_code,
    ed.offer_id,
    ed.customer_segment_id_test,
    ed.customer_segment_id_control,
    ed.discount,
    ed.start_date,
    ed.end_date,
    ed.value_discount,
    bs.addl_product_id AS card_id,
    bs2.name,
    bcat2.name AS cat,
    bcat.name as subcat
    
FROM exploded_data                                                                  ed
JOIN number_series                                                                  ns      ON ns.n <= regexp_count(ed.values_string, ',') + 1 AND trim(split_part(ed.values_string, ',', ns.n)) <> ''
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = trim(split_part(ed.values_string, ',', ns.n))::int
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"                bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat2   ON bcx.category_id = bcat2.category_id
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref"   bcs     ON bo.customer_segment_id = bcs.customer_segment_id

WHERE
    -- FIX SUPER DESCUENTOS
    bcat2.category_id NOT IN ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
"""#.format(clientes=tuple(set(clear_buyers[clear_buyers.registered_city == city].customer_id.unique())), city = city)

df_offers = read_connection_data_warehouse.runQuery(query)
df_offers['start_date'] = pd.to_datetime(df_offers['start_date'])
df_offers['end_date'] = pd.to_datetime(df_offers['end_date'])

In [34]:
df_offers = df_offers.loc[~df_offers.offer_id.isin([-14502687,-14502686])].reset_index(drop=True)

In [35]:
df_offers

,region_code,offer_id,customer_segment_id_test,customer_segment_id_control,discount,start_date,end_date,value_discount,card_id,name,cat,subcat
0,SPO,-14508101,157332,155136,HOOK_LOCALIZACION_SPO_296389_VLP_2023-05-02_SP...,2023-05-03,2023-05-10,7.0,190559,Café Pilão Tradicional Almofada 500g,Mercearia,"Café, Chocolates e Infusões"
1,SPO,-14508094,157474,157610,HOOK_LOCALIZACION_SPO_72488_GRU_2023-05-02_SPO...,2023-05-03,2023-05-10,5.0,421531,Queijo Mussarela 4kg - Litoral,Laticínios e Ovos,Queijos
2,SPO,-14508105,157412,155133,HOOK_LOCALIZACION_SPO_Arroz Branco Campeiro 5K...,2023-05-03,2023-05-10,8.5,231959,Arroz Branco Campeiro 5Kg,Mercearia,Arroz
3,SPO,-14508099,157608,157567,HOOK_LOCALIZACION_SPO_235489_STA_2023-05-02_SP...,2023-05-03,2023-05-10,3.5,59581,Feijão Carioca Camil 1Kg,Mercearia,Feijão
4,SPO,-14508104,155134,155135,HOOK_LOCALIZACION_SPO_Feijão Carioca Camil 1Kg...,2023-05-03,2023-05-10,8.5,231959,Arroz Branco Campeiro 5Kg,Mercearia,Arroz
5,SPO,-14508100,157566,157413,HOOK_LOCALIZACION_SPO_296389_GRU_2023-05-02_SP...,2023-05-03,2023-05-10,3.5,59581,Feijão Carioca Camil 1Kg,Mercearia,Feijão
6,SPO,-14508093,157653,157475,HOOK_LOCALIZACION_SPO_575574_STA_2023-05-02_SP...,2023-05-03,2023-05-10,5.0,421531,Queijo Mussarela 4kg - Litoral,Laticínios e Ovos,Queijos
7,SPO,-14508096,157284,157159,HOOK_LOCALIZACION_SPO_72488_STA_2023-05-02_SPO...,2023-05-03,2023-05-10,8.5,413151,Filé de Peito de Frango - Bello Alimentos,"Carnes, Aves e Peixes",Frango
8,SPO,-14508102,157606,157607,HOOK_LOCALIZACION_SPO_296389_STA_2023-05-02_SP...,2023-05-03,2023-05-10,7.0,190559,Café Pilão Tradicional Almofada 500g,Mercearia,"Café, Chocolates e Infusões"
9,SPO,-14508092,157333,157569,HOOK_LOCALIZACION_SPO_575574_VLP_2023-05-02_SP...,2023-05-03,2023-05-10,5.0,421531,Queijo Mussarela 4kg - Litoral,Laticínios e Ovos,Queijos


In [36]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (

SELECT DISTINCT
    bo.order_id,
    --ffg.close_date
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment" ba          ON ba.order_id = bo.order_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    AND bo.submit_date >= DATE_TRUNC('week', date(getdate()) - interval '3 week')
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{ciudad}'
    
--GROUP BY bo.order_id, ffg.close_date
)

SELECT DISTINCT
    s.site_identifier_value AS city,
    TO_CHAR(DATE_TRUNC('week', DATE_TRUNC('day', bo.submit_date) + INTERVAL '5 day'), 'WW') AS week,
    --TO_CHAR(bo.submit_date, 'WW') AS week,
    bo.customer_id,
    bs2.name,
    bs.addl_product_id as padre_sku_id,
    bcat2.name AS category,
    bcat.name AS subcat,
    MIN(bo.submit_date) AS min_date,
    COUNT(DISTINCT bo.order_id) as orders,
    CASE WHEN s.site_identifier_value = 'CMX' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'GDL' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'PBC' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'MTY' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'SPO' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
        WHEN s.site_identifier_value = 'BHZ' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
        WHEN s.site_identifier_value = 'CWB' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
        WHEN s.site_identifier_value = 'VCP' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
    ELSE SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/3776) END AS gmv_usd,
    CASE WHEN s.site_identifier_value = 'CMX' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
        WHEN s.site_identifier_value = 'GDL' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
        WHEN s.site_identifier_value = 'PBC' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
        WHEN s.site_identifier_value = 'MTY' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
        WHEN s.site_identifier_value = 'SPO' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
        WHEN s.site_identifier_value = 'BHZ' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
        WHEN s.site_identifier_value = 'CWB' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
        WHEN s.site_identifier_value = 'VCP' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
    ELSE (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/3776) END AS discount_usd,
    SUM(boi.quantity* foi.step_unit) as cant

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"     boipd   ON boipd.order_item_id=boi.order_item_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"       boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
    AND LOWER(boida.adjustment_reason) ILIKE '%hook%'
    
GROUP BY 1,2,3,4,5,6,7
""".format(ciudad=city_code)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd","discount_usd","cant"]] = data_ventas_aux[["gmv_usd","discount_usd","cant"]].astype(float)
#data_ventas["submit_date"] = pd.to_datetime(data_ventas["submit_date"])
#data_ventas_aux = data_ventas_aux.loc[data_ventas_aux.week.isin(['05','06','07'])].reset_index(drop=True)

In [37]:
data_ventas_aux.groupby('week').min_date.min()

week
16   2023-04-17 00:00:56.560
17   2023-04-19 00:00:12.475
18   2023-04-26 00:01:11.633
19   2023-05-03 00:00:11.076
20   2023-05-10 00:03:05.990
Name: min_date, dtype: datetime64[ns]

# Cleaning Data and some functions

In [38]:
if city_code == 'SPO':
    skus_exp = [231959, 190559, 59581, 413151, 421531]

In [39]:
df_exp = data_ventas_aux.loc[data_ventas_aux.padre_sku_id.isin(skus_exp)]

df_exp =  pd.merge(df_exp, df_margen2.loc[df_margen2.region == city_code].reset_index(drop=True),  how='left', left_on=['padre_sku_id'], right_on = ['id_tarjeta'])

df_exp['cash_margin'] = df_exp.gmv_usd * df_exp.GM

df_exp = df_exp.loc[:,['city','week','customer_id','name','padre_sku_id','category','subcat','orders','gmv_usd','discount_usd','cant','GM','cash_margin']]

In [40]:
all_weeks = data_ventas_aux.groupby('week').customer_id.nunique()

In [41]:
all_weeks

week
16     5163
17     8828
18     7911
19    11010
20      869
Name: customer_id, dtype: int64

In [42]:
def customers_offer(grupo):
    query = """
    SELECT DISTINCT 
        bcs.customer_segment_id, 
        bcs.external_identifier::int AS customer_id
    FROM postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref" bcs
    WHERE bcs.customer_segment_id = {clientes}
    """.format(clientes=grupo)

    df_clients = read_connection_data_warehouse.runQuery(query)
    return df_clients.customer_id.unique()

In [43]:
def df_info_filter_offer(df_offers_grouped, i, week, test, control):
    df_filter_test = df_exp[(df_exp['city'] == df_offers_grouped.loc[i,'region_code']) & 
                         (df_exp['customer_id'].isin(test)) & 
                         (df_exp['week'] == week) & 
                         (df_exp['padre_sku_id'] == df_offers_grouped.loc[i,'card_id'])].copy()
    df_filter_control = df_exp[(df_exp['city'] == df_offers_grouped.loc[i,'region_code']) & 
                         (df_exp['customer_id'].isin(control)) & 
                         (df_exp['week'] == week) & 
                         (df_exp['padre_sku_id'] == df_offers_grouped.loc[i,'card_id'])].copy()
    
    if len(df_filter_control) == 0:
        dic = {}
        dic = df_filter_test.iloc[0].to_dict()
        dic['week'] = week
        dic['customer_id'] = 0
        dic['gmv_usd'] = 0
        dic['discount_usd'] = 0
        dic['cant'] = 0
        new_df = pd.DataFrame(dic, index=[0])
        df_filter_control = pd.concat([df_filter_control, new_df], ignore_index=True)

    if len(df_filter_test) == 0:
        dic = {}
        dic = df_filter_control.iloc[0].to_dict()
        dic['week'] = week
        dic['customer_id'] = 0
        dic['gmv_usd'] = 0
        dic['discount_usd'] = 0
        dic['cant'] = 0
        new_df = pd.DataFrame(dic, index=[0])
        df_filter_test = pd.concat([df_filter_test, new_df], ignore_index=True)
        
    df_filter_test['group'] = 'test'
    df_filter_control['group'] = 'control'
    df_filter_test['all_customer'] = len(test)
    df_filter_control['all_customer'] = len(control)
    df_filter = pd.concat([df_filter_test, df_filter_control]).reset_index(drop=True)
    df_filter['offer_id'] = df_offers_grouped.loc[i,'offer_id']
    df_filter['GM'] = df_filter.padre_sku_id.apply(lambda x: 0.1 if len(df_margen2.loc[df_margen2.id_tarjeta == x]['GM'].unique()) == 0 else df_margen2.loc[df_margen2.id_tarjeta == x]['GM'].unique()[0])
    #df_filter['GM'] = df_filter.padre_sku_id.apply(lambda x: df_margen2.loc[df_margen2.id_tarjeta == x]['GM'].unique()[0])
    df_filter['cash_margin'] = df_filter.gmv_usd * df_filter.GM
    df_filter['net_cash_margin'] = df_filter.cash_margin - df_filter.discount_usd
    return df_filter

In [44]:
df_exp=df_exp.loc[df_exp.week.isin(['16','17','18','19'])]

In [45]:
results_df = pd.DataFrame()
for i in range(len(df_offers)):
    print(df_offers.loc[i,'offer_id'])
    test = customers_offer(df_offers.loc[i,'customer_segment_id_test'])
    control = customers_offer(df_offers.loc[i,'customer_segment_id_control'])
    for week in df_exp.week.unique():
        info_offer = df_info_filter_offer(df_offers, i, week, test, control)
        results_offer = (info_offer.groupby(by=['city', 'week','offer_id', 'name', 'group', 'all_customer'])
                            .agg(customer_count=('customer_id', 'nunique'),
                                gmv_usd=('gmv_usd', np.sum),
                                discount_usd=('discount_usd', np.sum),
                                quantity=('cant', np.sum),
                                margen=('GM', np.mean),
                                cash_margin=('cash_margin', np.sum),
                                net_cash_margin=('net_cash_margin', np.sum))
                            .reset_index()
                )
    
        results_offer['convertion_rate'] = results_offer.customer_count / results_offer.all_customer       
        results_offer['AOV'] = results_offer.gmv_usd / results_offer.customer_count
        results_offer['AOV_GM'] = results_offer.cash_margin / results_offer.customer_count
        results_offer['dct_avg'] = results_offer.discount_usd / results_offer.customer_count
        
        results_df = pd.concat([results_df, results_offer]).reset_index(drop=True)

-14508101
-14508094
-14508105
-14508099
-14508104
-14508100
-14508093
-14508096
-14508102
-14508092
-14508095
-14508097
-14508103
-14508098
-14508106


In [46]:
import re

pattern = re.compile(r'_(STA|GRU|VLP)_')
# extract the warehouse name from the column and create a new column in the dataframe
df_offers['wh'] = df_offers['discount'].apply(lambda x: pattern.search(x).group(1))

In [47]:
results_df = pd.merge(results_df, df_offers[['offer_id','wh']],  how='left', left_on=['offer_id'], right_on = ['offer_id'])

In [48]:
results_df.name.unique()

array(['Café Pilão Tradicional Almofada 500g',
       'Queijo Mussarela 4kg - Litoral', 'Arroz Branco Campeiro 5Kg',
       'Feijão Carioca Camil 1Kg',
       'Filé de Peito de Frango - Bello Alimentos'], dtype=object)

In [63]:
results_df.loc[(results_df.name == 'Café Pilão Tradicional Almofada 500g') &  (results_df.wh.isin(['GRU','VLP','STA'])) & (results_df.week == '19')]

,city,week,offer_id,name,group,all_customer,customer_count,gmv_usd,discount_usd,quantity,margen,cash_margin,net_cash_margin,convertion_rate,AOV,AOV_GM,dct_avg,wh,prueba
2,SPO,19,-14508101,Café Pilão Tradicional Almofada 500g,control,199,3,119.28419,7.47788,31.0,0.162185,19.346090,11.868210,0.015075,39.761397,6.448697,2.492627,VLP,0.062690
3,SPO,19,-14508101,Café Pilão Tradicional Almofada 500g,test,200,15,615.76833,42.91781,160.0,0.162185,99.868304,56.950494,0.075000,41.051222,6.657887,2.861187,VLP,0.069698
66,SPO,19,-14508102,Café Pilão Tradicional Almofada 500g,control,237,2,35.05262,2.06315,9.0,0.162185,5.685004,3.621854,0.008439,17.526310,2.842502,1.031575,STA,0.058859
67,SPO,19,-14508102,Café Pilão Tradicional Almofada 500g,test,238,17,546.86303,38.20623,142.0,0.162185,88.692908,50.486678,0.071429,32.168414,5.217230,2.247425,STA,0.069864
98,SPO,19,-14508103,Café Pilão Tradicional Almofada 500g,control,143,5,135.05260,7.21682,35.0,0.162185,21.903488,14.686668,0.034965,27.010520,4.380698,1.443364,GRU,0.053437
99,SPO,19,-14508103,Café Pilão Tradicional Almofada 500g,test,143,16,374.12623,26.16412,97.0,0.162185,60.677613,34.513493,0.111888,23.382889,3.792351,1.635258,GRU,0.069934


In [64]:
results_df.loc[(results_df.name == 'Café Pilão Tradicional Almofada 500g') &  (results_df.wh.isin(['GRU','VLP','STA'])) & (results_df.week == '18')]

,city,week,offer_id,name,group,all_customer,customer_count,gmv_usd,discount_usd,quantity,margen,cash_margin,net_cash_margin,convertion_rate,AOV,AOV_GM,dct_avg,wh,prueba
4,SPO,18,-14508101,Café Pilão Tradicional Almofada 500g,control,199,12,257.66729,12.87574,66.0,0.162185,41.789735,28.913995,0.060302,21.472274,3.482478,1.072978,VLP,0.049970
5,SPO,18,-14508101,Café Pilão Tradicional Almofada 500g,test,200,13,609.11571,30.47573,156.0,0.162185,98.789350,68.313620,0.065000,46.855055,7.599181,2.344287,VLP,0.050033
68,SPO,18,-14508102,Café Pilão Tradicional Almofada 500g,control,237,12,176.04627,8.78734,45.0,0.162185,28.552041,19.764701,0.050633,14.670522,2.379337,0.732278,STA,0.049915
69,SPO,18,-14508102,Café Pilão Tradicional Almofada 500g,test,238,16,273.59994,13.66097,70.0,0.162185,44.373770,30.712800,0.067227,17.099996,2.773361,0.853811,STA,0.049930
100,SPO,18,-14508103,Café Pilão Tradicional Almofada 500g,control,143,12,232.87993,11.63363,60.0,0.162185,37.769600,26.135970,0.083916,19.406661,3.147467,0.969469,GRU,0.049955
101,SPO,18,-14508103,Café Pilão Tradicional Almofada 500g,test,143,14,302.36623,15.10308,78.0,0.162185,49.039227,33.936147,0.097902,21.597588,3.502802,1.078791,GRU,0.049950


In [51]:
results_df["prueba"] = results_df.dct_avg/results_df.AOV